# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [30]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from time import sleep
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
df= pd.read_csv("output_data.csv")
df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,komsomolskiy,40.4272,71.7189,16.38,25.0,70.0,0.00,UZ,1.635224e+09
1,1,butaritari,3.0707,172.7902,28.18,75.0,9.0,3.61,KI,1.635224e+09
2,2,goderich,43.7501,-81.7165,6.21,81.0,100.0,7.20,CA,1.635224e+09
3,3,mar del plata,-38.0023,-57.5575,13.20,80.0,0.0,1.34,AR,1.635224e+09
4,4,sisophon,13.5859,102.9737,28.57,63.0,83.0,3.37,KH,1.635224e+09
...,...,...,...,...,...,...,...,...,...,...
521,578,abohar,30.1500,74.1833,24.54,37.0,0.0,2.27,IN,1.635225e+09
522,579,notse,6.9500,1.1667,23.44,94.0,100.0,1.60,TG,1.635225e+09
523,580,solhan,38.9634,41.0293,3.79,47.0,43.0,1.84,TR,1.635225e+09
524,581,nishihara,26.1842,127.7558,26.28,52.0,20.0,4.63,JP,1.635225e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
locations = df[['Lat','Lng']]
figure_layout = {
    'width': '1000px',
    'height': '800px',
    'border': '2px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}


In [12]:
fig = gmaps.figure(layout=figure_layout)
locations = df[["Lat", "Lng"]]
weight = df["Humidity"]
fig.add_layer(gmaps.heatmap_layer(locations, weights=weight, max_intensity = 100))
fig

Figure(layout=FigureLayout(border='2px solid black', height='800px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
#Temp between 70 and 80 F (21 C to 27 C)
#Wind < 10 mph
#Zero Cloudiness
condition1 = (df['Max Temp'] < 27) & (df['Max Temp'] > 21)
condition2 = df['Wind Speed'] < 10
condition3 = df['Cloudiness'] == 0
hoteldf = df.loc[condition1 & condition2 & condition3,:]
hoteldf

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
57,62,dharchula,29.8500,80.5333,21.30,35.0,0.0,2.33,IN,1.635224e+09
94,106,saint-pierre,-21.3393,55.4781,25.94,63.0,0.0,8.75,RE,1.635224e+09
97,109,umm lajj,25.0213,37.2685,26.72,48.0,0.0,0.58,SA,1.635224e+09
158,179,qeshm,26.9581,56.2719,23.93,67.0,0.0,4.55,IR,1.635224e+09
197,224,arlit,18.7369,7.3853,26.36,13.0,0.0,2.68,NE,1.635224e+09
240,270,ghanzi,-21.5667,21.7833,22.08,49.0,0.0,4.20,BW,1.635224e+09
323,360,sitio novo do tocantins,-5.5183,-47.4611,26.96,94.0,0.0,0.00,BR,1.635224e+09
343,381,seminole,28.7086,-81.2081,23.76,94.0,0.0,3.67,US,1.635224e+09
521,578,abohar,30.1500,74.1833,24.54,37.0,0.0,2.27,IN,1.635225e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
hoteldf["Hotel Name"] = ""

<ipython-input-20-6ea2557b8a3c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hoteldf["Hotel Name"] = ""


In [21]:
hoteldf

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
57,62,dharchula,29.8500,80.5333,21.30,35.0,0.0,2.33,IN,1.635224e+09,
94,106,saint-pierre,-21.3393,55.4781,25.94,63.0,0.0,8.75,RE,1.635224e+09,
97,109,umm lajj,25.0213,37.2685,26.72,48.0,0.0,0.58,SA,1.635224e+09,
158,179,qeshm,26.9581,56.2719,23.93,67.0,0.0,4.55,IR,1.635224e+09,
197,224,arlit,18.7369,7.3853,26.36,13.0,0.0,2.68,NE,1.635224e+09,
240,270,ghanzi,-21.5667,21.7833,22.08,49.0,0.0,4.20,BW,1.635224e+09,
323,360,sitio novo do tocantins,-5.5183,-47.4611,26.96,94.0,0.0,0.00,BR,1.635224e+09,
343,381,seminole,28.7086,-81.2081,23.76,94.0,0.0,3.67,US,1.635224e+09,
521,578,abohar,30.1500,74.1833,24.54,37.0,0.0,2.27,IN,1.635225e+09,


In [32]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?parameters"
params = {'rankby': 'distance',
         'keyword': 'hotel',
         'key': g_key}
for index, row in hoteldf.iterrows():
    params['location'] = f"{row['Lat']}, {row['Lng']}"
    result = requests.get(base_url, params=params).json()
    hoteldf.loc[index, 'Hotel Name'] = result['results'][0]['name']
    print(index, result['results'][0]['name'])

C:\Users\deant\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


57 Himtop Hotel Dharchula
94 Hotel l'Endormi
97 Juman Hotel
158 Apadana Hotel
197 Hôtel Telwa Bungalow
240 Dqae Qare San Lodge
323 Borges Hotel
343 Best Western Plus Sanford Airport/Lake Mary Hotel
521 Piccadilly Hotel


In [33]:
hoteldf

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
57,62,dharchula,29.8500,80.5333,21.30,35.0,0.0,2.33,IN,1.635224e+09,Himtop Hotel Dharchula
94,106,saint-pierre,-21.3393,55.4781,25.94,63.0,0.0,8.75,RE,1.635224e+09,Hotel l'Endormi
97,109,umm lajj,25.0213,37.2685,26.72,48.0,0.0,0.58,SA,1.635224e+09,Juman Hotel
158,179,qeshm,26.9581,56.2719,23.93,67.0,0.0,4.55,IR,1.635224e+09,Apadana Hotel
197,224,arlit,18.7369,7.3853,26.36,13.0,0.0,2.68,NE,1.635224e+09,Hôtel Telwa Bungalow
240,270,ghanzi,-21.5667,21.7833,22.08,49.0,0.0,4.20,BW,1.635224e+09,Dqae Qare San Lodge
323,360,sitio novo do tocantins,-5.5183,-47.4611,26.96,94.0,0.0,0.00,BR,1.635224e+09,Borges Hotel
343,381,seminole,28.7086,-81.2081,23.76,94.0,0.0,3.67,US,1.635224e+09,Best Western Plus Sanford Airport/Lake Mary Hotel
521,578,abohar,30.1500,74.1833,24.54,37.0,0.0,2.27,IN,1.635225e+09,Piccadilly Hotel


In [35]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hoteldf.iterrows()]
hotellocations = hoteldf[["Lat", "Lng"]]

In [37]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(hotellocations)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(border='2px solid black', height='800px', margin='0 auto 0 auto', padding='1px', wi…